<a href="https://colab.research.google.com/github/veliona/music-streaming-platforms-NLP/blob/master/Spotify_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from collections import defaultdict
import json
import pandas as pd
from datetime import datetime
import requests
import os
import csv
import io
import sys
from google.colab import drive

In [20]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [21]:
sys.path.append('/content/gdrive/My Drive/Studia')
os.listdir('/content/gdrive/My Drive/Studia')

['Głos 008.m4a',
 'Głos 009.m4a',
 'Statystyka (1)',
 'MIG1.m4a',
 'LUMINO.xlsx',
 'Marketing',
 'E-sylabusy',
 'IT',
 'ETIX',
 'Statystyka',
 'Zarządzanie',
 'Lista nr 1.xls',
 '~$Etix.docx',
 '~$ix-procesy.docx',
 'Case studies book.pdf',
 'MSG',
 'podanie-o-urlop-dziekanski (1).doc',
 'r-spotify-Hot 04-23-2020.csv',
 'df_google_play_spotify.csv']

In [0]:
df_spotify_hot_posts = pd.read_csv('/content/gdrive/My Drive/Studia/r-spotify-Hot 04-23-2020.csv')

In [23]:
df_spotify_hot_posts.head()

,Title,Flair,Date Created,Upvotes,Upvote Ratio,ID,Edited?,Is Locked?,NSFW?,Is Spoiler?,Stickied?,URL,Comment Count,Text
0,Update: Upcoming playlist submission changes,NaN,04-10-2020 10:10:49,91,0.92,fy3tom,False,False,False,False,True,https://www.reddit.com/r/spotify/comments/fy3t...,51,"Hello everyone,\n\nAs a new moderator for this..."
1,Playlists Megathread - Submit here your playli...,Playlist,04-12-2020 11:59:36,47,0.84,fznwiy,04-12-2020 04:13:40,False,False,False,True,https://www.reddit.com/r/spotify/comments/fznw...,539,"As mentioned previously, submission of playlis..."
2,Seems like they removed All Eyez On Me,Other,04-23-2020 18:55:21,149,0.93,g6j9fg,False,False,False,False,False,https://www.reddit.com/r/spotify/comments/g6j9...,59,I can't listen to all eyez on me from tupac an...
3,"No more 2pac on spotify. Today, spotify put an...",Complaint,04-23-2020 21:44:36,50,0.81,g6l6m4,False,False,False,False,False,https://www.reddit.com/r/spotify/comments/g6l6...,27,NaN
4,Why is spotify still alive and big if it’s rou...,Question,04-23-2020 06:54:53,295,0.95,g697xy,04-23-2020 01:09:53,False,False,False,False,https://www.reddit.com/r/spotify/comments/g697...,76,I am aware that Spotify has finally made a pro...


In [0]:
df_spotify_hot_posts['title_content'] = df_spotify_hot_posts['Title'] + df_spotify_hot_posts['Text']

In [25]:
import nltk; nltk.download('stopwords')
! pip install -U spacy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [26]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
! pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [0]:
# Convert to list
list_text = df_spotify_hot_posts.title_content.values.tolist()

In [0]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(list_text))

In [30]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['update', 'upcoming', 'playlist', 'submission', 'changeshello', 'everyone', 'as', 'new', 'moderator', 'for', 'this', 'sub', 've', 'been', 'doing', 'some', 'minor', 'adjustments', 'to', 'try', 'more', 'effectively', 'curbing', 'the', 'constant', 'submissions', 'by', 'spammers', 'one', 'of', 'the', 'things', 'which', 'had', 'come', 'up', 'and', 'was', 'also', 'noted', 'in', 'the', 'thread', 'about', 'problems', 'here', 'is', 'handling', 'playlist', 'submissions', 'in', 'better', 'way', 'to', 'that', 'end', 'there', 'will', 'be', 'dedicated', 'thread', 'where', 'playlists', 'will', 'have', 'to', 'be', 'submitted', 'if', 'there', 'is', 'interest', 'we', 'may', 'see', 'about', 'splitting', 'out', 'to', 'small', 'degree', 'by', 'some', 'criteria', 'such', 'as', 'genre', 'new', 'artists', 'etc', 'anticipate', 'finalizing', 'the', 'automoderator', 'rules', 'within', 'the', 'next', 'few', 'days', 'and', 'then', 'blocking', 'any', 'playlist', 'submissions', 'outside', 'of', 'the', 'designated',

In [0]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [32]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['upcoming', 'playlist', 'submission', 'changeshello', 'new', 'moderator', 'sub', 'minor', 'adjustment', 'try', 'effectively', 'curb', 'constant', 'submission', 'spammer', 'thing', 'come', 'also', 'note', 'thread', 'problem', 'handle', 'playlist', 'submission', 'well', 'way', 'end', 'dedicated', 'thread', 'playlist', 'submit', 'interest', 'may', 'see', 'splitting', 'small', 'degree', 'criterion', 'genre', 'new', 'artist', 'automoderator', 'rule', 'next', 'day', 'block', 'playlist', 'submission', 'designate', 'thread', 'would', 'also', 'like', 'standardized', 'way', 'format', 'description', 'possibly', 'require', 'tag', 'genre', 'allow', 'easy', 'search', 'personally', 'think', 'submitter', 'heavy', 'handed', 'emojis', 'title', 'idea', 'suggestion', 'area', 'let', 'know', 'thank']]


In [33]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 1), (38, 4), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 4), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 3), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1)]]


In [0]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [35]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.085*"spotify" + 0.046*"music" + 0.037*"feature" + 0.026*"know" + '
  '0.025*"would" + 0.022*"get" + 0.022*"need" + 0.020*"sometimes" + '
  '0.017*"lyric" + 0.017*"order"'),
 (1,
  '0.108*"song" + 0.052*"play" + 0.051*"artist" + 0.041*"playlist" + '
  '0.032*"album" + 0.024*"go" + 0.023*"listen" + 0.023*"make" + 0.018*"sort" + '
  '0.018*"ad"'),
 (2,
  '0.044*"put" + 0.040*"change" + 0.039*"account" + 0.034*"password" + '
  '0.031*"can" + 0.027*"open" + 0.025*"playlist" + 0.025*"people" + '
  '0.021*"friend" + 0.020*"probably"'),
 (3,
  '0.107*"song" + 0.047*"back" + 0.034*"playlist" + 0.028*"go" + 0.027*"play" '
  '+ 0.025*"add" + 0.022*"fix" + 0.022*"listen" + 0.022*"like" + '
  '0.021*"different"'),
 (4,
  '0.039*"allow" + 0.037*"next" + 0.035*"finish" + 0.029*"remember" + '
  '0.027*"send" + 0.024*"band" + 0.023*"stream" + 0.019*"couple" + '
  '0.017*"apply" + 0.016*"already"'),
 (5,
  '0.044*"playlist" + 0.033*"mix" + 0.031*"hear" + 0.030*"able" + '
  '0.030*"country" + 0

In [36]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.998772175344449

Coherence Score:  0.3483922809458543


In [37]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.306303  0.084441       1        1  15.841595
13     0.219485 -0.153848       2        1  15.394849
0      0.136940 -0.158351       3        1   9.799237
3      0.238932  0.140163       4        1   8.645665
12     0.052132  0.115885       5        1   5.407002
2      0.020902  0.121230       6        1   4.562370
16     0.034174  0.023783       7        1   4.155786
6     -0.070813 -0.041715       8        1   3.960296
9     -0.067993 -0.079103       9        1   3.746403
11    -0.012302 -0.154729      10        1   3.545021
8     -0.109102 -0.072838      11        1   3.506572
14     0.006052 -0.079851      12        1   3.385389
7     -0.088040 -0.095925      13        1   3.276556
19    -0.068879  0.080390      14        1   2.777321
15    -0.038376 -0.078216      15        1   2.763014
5     -0.042150  0.085085      16        1   2.411294
10    -0.123233  0.081649      17        1   2.234194
18    -0.146262  0.021756      18        1   1.697382
4     -0.119022  0.057107      19        1   1.544097
17    -0.128746  0.103086      20        1   1.345960, topic_info=         Term        Freq       Total Category  logprob  loglift
110      song  347.000000  347.000000  Default  30.0000  30.0000
38   playlist  233.000000  233.000000  Default  29.0000  29.0000
208   account   89.000000   89.000000  Default  28.0000  28.0000
160   spotify  388.000000  388.000000  Default  27.0000  27.0000
4      artist  151.000000  151.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
142   finally    2.498221    5.974096  Topic20  -4.2394   3.4362
90       make    3.461694   84.358307  Topic20  -3.9132   1.1147
172       bad    2.349983   18.090410  Topic20  -4.3006   2.2671
147      keep    2.244217   41.620003  Topic20  -4.3466   1.3878
309      sign    1.760193   14.637736  Topic20  -4.5896   2.1899

[977 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
580       3  0.910711     ability
245       1  0.669879        able
245      16  0.274042        able
245      18  0.030449        able
337      14  0.890135  absolutely
...     ...       ...         ...
130      12  0.186026        year
728       6  0.836568   yesterday
295       3  0.485244         yet
295       6  0.485244         yet
1774      2  0.909158     youtube

[1257 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 14, 1, 4, 13, 3, 17, 7, 10, 12, 9, 15, 8, 20, 16, 6, 11, 19, 5, 18])

## Hyperparameters tunning

In [0]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [0]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 10
max_topics = 20
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]
corpus_title = ['75% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        for a in alpha:
            # iterare through beta values
            for b in beta:
                # get the coherence score for the given parameters
                cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                              k=k, a=a, b=b)
                # Save the model results
                model_results['Validation_Set'].append(corpus_title)
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
                
                pbar.update(1)
pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
pbar.close()

  0%|          | 0/540 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
  9%|▉         | 48/540 [03:20<33:09,  4.04s/it]

## Final model

In [0]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                      id2word=id2word,
                                      num_topics=7, 
                                      random_state=100,
                                      chunksize=100,
                                      passes=10,
                                      alpha= 0.01,
                                      eta=0.9)

In [0]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [0]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

# Google Play Spotify

In [0]:
df_google_play_spotify = pd.read_csv('/content/gdrive/My Drive/Studia/df_google_play_spotify.csv')

In [5]:
df_google_play_spotify.head()

,Unnamed: 0,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,0,Frank Helman,https://lh3.googleusercontent.com/a-/AOh14Gjpe...,No complaints. Great price options,5,0,8.5.51.941,2020-04-24 21:12:18,NaN,NaN
1,1,Harshavardhana vc moorthy,https://lh3.googleusercontent.com/a-/AOh14Gj1H...,The best app for music 👌. Spotify stands up fo...,5,0,8.5.54.869,2020-04-24 21:12:03,NaN,NaN
2,2,Nadine Burns,https://lh3.googleusercontent.com/-kLLhhsAK7Jw...,It was good but it stop working 😐😑😲👎👎👎👎👎🖓🖓🖓🖓,3,0,NaN,2020-04-24 21:11:33,NaN,NaN
3,3,Roland Cardinal,https://lh3.googleusercontent.com/-yowtW4adbO4...,Really love it,5,0,7.9.0.746,2020-04-24 21:09:36,NaN,NaN
4,4,John Laxey,https://lh3.googleusercontent.com/a-/AOh14GgDM...,One of the best music apps to use!!,5,0,8.5.54.869,2020-04-24 21:07:25,NaN,NaN


In [6]:
df_google_play_spotify.tail()

,Unnamed: 0,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
95,95,robin maxwells,https://lh3.googleusercontent.com/a-/AOh14GiTC...,Awful. It randomly starts playing songs from a...,1,0,8.5.54.869,2020-04-24 19:46:13,NaN,NaN
96,96,Hailey Rose,https://lh3.googleusercontent.com/-FRXnWto8ntA...,I love this app but sometimes it let down. But...,5,0,8.5.54.869,2020-04-24 19:44:13,NaN,NaN
97,97,Ian Reed,https://lh3.googleusercontent.com/-TLDDBIsHEb8...,April 2020 update brings a new set of bugs. Ke...,2,0,8.5.54.869,2020-04-24 19:44:13,NaN,NaN
98,98,Ayla Smith,https://lh3.googleusercontent.com/-2HeW47LSymw...,So much fun to use and listen to. Love it the ...,4,0,8.5.54.869,2020-04-24 19:43:21,NaN,NaN
99,99,Mark Pugh,https://lh3.googleusercontent.com/-XLnehI9-kNo...,Just awful There're much better media players ...,1,0,8.5.54.869,2020-04-24 19:42:17,NaN,NaN
